In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import math
import re
from PIL import Image
import seaborn as sns
from numpy import var

In [2]:
# get the list of all files in the folder - "train" and "val and test"
from os import listdir
from os.path import isfile, join
mypath = "F:\\Machine Learning\\Assignments\\Capstone Project - Bird Classification\\Bird_classification\\train"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f)) and re.findall(r'JPG',f)]

mypath_test = "F:\\Machine Learning\\Assignments\\Capstone Project - Bird Classification\\Bird_classification\\val_and_test"
onlyfiles_test = [f for f in listdir(mypath_test) if isfile(join(mypath_test, f)) and re.findall(r'JPG',f)]

In [3]:
onlyfiles[1]

'100_1.JPG'

In [4]:
onlyfiles_test[1]

'0001.JPG'

In [5]:
df = pd.DataFrame(columns = ['File','Covariance_l','Correlation_l','Var(x)_l','Var(y)_l','Angle1_l','Angle2_l',                            
                                    'Covariance_c','Correlation_c','Var(x)_c','Var(y)_c','Angle1_c','Angle2_c',
                                    'Covariance_sx','Correlation_sx','Var(x)_sx','Var(y)_sx','Angle1_sx','Angle2_sx',
                                    'Covariance_sy','Correlation_sy','Var(x)_sy','Var(y)_sy','Angle1_sy','Angle2_sy',                                  
                             'BIRD'])
df

,File,Covariance_l,Correlation_l,Var(x)_l,Var(y)_l,Angle1_l,Angle2_l,Covariance_c,Correlation_c,Var(x)_c,...,Var(y)_sx,Angle1_sx,Angle2_sx,Covariance_sy,Correlation_sy,Var(x)_sy,Var(y)_sy,Angle1_sy,Angle2_sy,BIRD


In [6]:
def filter_value(type):
    
    if type=='laplacian':
        value = cv2.Laplacian(gray,cv2.CV_64F)
    elif type == 'canny':
        value = cv2.Canny(gray,100,200)
    elif type == 'sobelx':
        value = cv2.Sobel(gray,cv2.CV_64F,0,1,ksize=5)   
    elif type == 'sobely':
        value = cv2.Sobel(gray,cv2.CV_64F,1,0,ksize=5)
    return value

In [7]:
def edge_detect(gray,type):
   
        value = filter_value(type)           
        _,thresh = cv2.threshold(value,15,255,cv2.THRESH_BINARY)            
        x,y = np.nonzero(thresh)            
        x = x - np.mean(x)
        y = y - np.mean(y)            
        coords = np.vstack([x, y])          
        cov = np.cov(coords)            
        if np.any(np.isnan(cov)) == True:
            cov.fill(0)           
        evals, evecs = np.linalg.eig(cov)
        sort_indices = np.argsort(evals)[::-1]
        x_v1, y_v1 = evecs[:, sort_indices[0]]  
        x_v2, y_v2 = evecs[:, sort_indices[1]]
        angle1 = math.degrees(np.arctan(y_v1/x_v1))
        angle2 = math.degrees(np.arctan(y_v2/x_v2))   
        
        return np.cov(x,y)[0][1],np.corrcoef(x,y)[0][1],var(x),var(y),angle1,angle2
    
    

In [8]:
for i in range(len(onlyfiles)):    
    
    try:
        file = onlyfiles[i]
        img = cv2.imread('train/'+file)     
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)              
        cov_l,cor_l,varx_l,vary_l,angle1_l,angle2_l = edge_detect(gray,'laplacian') 
        cov_c,cor_c,varx_c,vary_c,angle1_c,angle2_c = edge_detect(gray,'canny')
        cov_sx,cor_sx,varx_sx,vary_sx,angle1_sx,angle2_sx = edge_detect(gray,'sobelx')   
        cov_sy,cor_sy,varx_sy,vary_sy,angle1_sy,angle2_sy = edge_detect(gray,'sobely')       
        bird = re.findall(r'\d',onlyfiles[i])[-1]
        df = df.append({'File':file,'Covariance_l':cov_l,'Correlation_l':cor_l,'Var(x)_l':varx_l,'Var(y)_l':vary_l,'Angle1_l':angle1_l,'Angle2_l':angle2_l,                            
                                    'Covariance_c':cov_c,'Correlation_c':cor_c,'Var(x)_c':varx_c,'Var(y)_c':vary_c,'Angle1_c':angle1_c,'Angle2_c':angle2_c,
                                    'Covariance_sx':cov_sx,'Correlation_sx':cor_sx,'Var(x)_sx':varx_sx,'Var(y)_sx':vary_sx,'Angle1_sx':angle1_sx,'Angle2_sx':angle2_sx,
                                    'Covariance_sy':cov_sy,'Correlation_sy':cor_sy,'Var(x)_sy':varx_sy,'Var(y)_sy':vary_sy,'Angle1_sy':angle1_sy,'Angle2_sy':angle2_sy,                                    
                        'BIRD':bird},ignore_index=True)
                     
    except:
        print('some error at: ',i)
        continue

C:\Users\USER\AppData\Roaming\Python\Python36\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\USER\AppData\Roaming\Python\Python36\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\USER\AppData\Roaming\Python\Python36\site-packages\numpy\lib\function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
C:\Users\USER\AppData\Roaming\Python\Python36\site-packages\numpy\core\_methods.py:78: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: Degrees of freedom <= 0 for slice
  if __name__ == '__main__':
C:\Users\USER\AppData\Roaming\Python\Python36\site-packages\numpy\lib\function_base.py:2451: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divid

In [9]:
df.head(2)

,File,Covariance_l,Correlation_l,Var(x)_l,Var(y)_l,Angle1_l,Angle2_l,Covariance_c,Correlation_c,Var(x)_c,...,Var(y)_sx,Angle1_sx,Angle2_sx,Covariance_sy,Correlation_sy,Var(x)_sy,Var(y)_sy,Angle1_sy,Angle2_sy,BIRD
0,100_0.JPG,20.100772,0.004853,4048.689360,4224.828254,83.580110,-6.419890,NaN,NaN,NaN,...,4114.828432,-81.560078,8.439922,10.163191,0.002455,4171.049257,4108.540380,9.006279,-80.993721,0
1,100_1.JPG,-1141.668144,-0.429730,3473.922202,2020.861100,-28.729291,61.270709,-1140.384774,-0.49027,3136.6043,...,4159.928254,-7.021329,82.978671,-184.347678,-0.044806,3983.028610,4249.555354,-62.932006,27.067994,1


In [10]:
#to save to csv file
df.to_csv('Bird_df.csv')

In [11]:
#to make empty
df = df[0:0]

In [12]:
df

,File,Covariance_l,Correlation_l,Var(x)_l,Var(y)_l,Angle1_l,Angle2_l,Covariance_c,Correlation_c,Var(x)_c,...,Var(y)_sx,Angle1_sx,Angle2_sx,Covariance_sy,Correlation_sy,Var(x)_sy,Var(y)_sy,Angle1_sy,Angle2_sy,BIRD


In [13]:
df_test = pd.DataFrame(columns = ['File','Covariance_l','Correlation_l','Var(x)_l','Var(y)_l','Angle1_l','Angle2_l',                            
                                    'Covariance_c','Correlation_c','Var(x)_c','Var(y)_c','Angle1_c','Angle2_c',
                                    'Covariance_sx','Correlation_sx','Var(x)_sx','Var(y)_sx','Angle1_sx','Angle2_sx',
                                    'Covariance_sy','Correlation_sy','Var(x)_sy','Var(y)_sy','Angle1_sy','Angle2_sy',                                    
                             'BIRD'])
df_test

,File,Covariance_l,Correlation_l,Var(x)_l,Var(y)_l,Angle1_l,Angle2_l,Covariance_c,Correlation_c,Var(x)_c,...,Var(y)_sx,Angle1_sx,Angle2_sx,Covariance_sy,Correlation_sy,Var(x)_sy,Var(y)_sy,Angle1_sy,Angle2_sy,BIRD


In [14]:
for i in range(len(onlyfiles_test)):    
    
    try:
        file = onlyfiles_test[i]
        img = cv2.imread('val_and_test/'+file)     
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)              
        cov_l,cor_l,varx_l,vary_l,angle1_l,angle2_l = edge_detect(gray,'laplacian') 
        cov_c,cor_c,varx_c,vary_c,angle1_c,angle2_c = edge_detect(gray,'canny')
        cov_sx,cor_sx,varx_sx,vary_sx,angle1_sx,angle2_sx = edge_detect(gray,'sobelx')   
        cov_sy,cor_sy,varx_sy,vary_sy,angle1_sy,angle2_sy = edge_detect(gray,'sobely')          
        df_test = df_test.append({'File':file,
                                  'Covariance_l':cov_l,'Correlation_l':cor_l,'Var(x)_l':varx_l,'Var(y)_l':vary_l,'Angle1_l':angle1_l,'Angle2_l':angle2_l,                            
                                  'Covariance_c':cov_c,'Correlation_c':cor_c,'Var(x)_c':varx_c,'Var(y)_c':vary_c,'Angle1_c':angle1_c,'Angle2_c':angle2_c,
                                  'Covariance_sx':cov_sx,'Correlation_sx':cor_sx,'Var(x)_sx':varx_sx,'Var(y)_sx':vary_sx,'Angle1_sx':angle1_sx,'Angle2_sx':angle2_sx,
                                  'Covariance_sy':cov_sy,'Correlation_sy':cor_sy,'Var(x)_sy':varx_sy,'Var(y)_sy':vary_sy,'Angle1_sy':angle1_sy,'Angle2_sy':angle2_sy                                  
                             },ignore_index=True)
                     
    except:
        print('some error at: ',i)
        continue

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: Degrees of freedom <= 0 for slice
  if __name__ == '__main__':
C:\Users\USER\AppData\Roaming\Python\Python36\site-packages\numpy\lib\function_base.py:2451: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
C:\Users\USER\AppData\Roaming\Python\Python36\site-packages\numpy\lib\function_base.py:2451: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in double_scalars
  app.launch_new_instance()
C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: Degrees of freedom <= 0 for slice
C:\Users\USER\AppData\Roaming\Python\Python36\site-packages\numpy\lib\function_base.py:2522: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
C:\Users\USER\AppData\Roaming\Python\Python36\site-p

In [15]:
df_test.head(2)

,File,Covariance_l,Correlation_l,Var(x)_l,Var(y)_l,Angle1_l,Angle2_l,Covariance_c,Correlation_c,Var(x)_c,...,Var(y)_sx,Angle1_sx,Angle2_sx,Covariance_sy,Correlation_sy,Var(x)_sy,Var(y)_sy,Angle1_sy,Angle2_sy,BIRD
0,0000.JPG,-119.309524,-0.111133,1826.252626,617.309262,-5.522878,84.477122,-156.516393,-0.765936,318.392398,...,4152.939747,-46.530068,43.469932,26.027402,0.006273,4186.850480,4110.813800,17.197175,-72.802825,NaN
1,0001.JPG,NaN,NaN,0.000000,0.000000,90.000000,0.000000,NaN,NaN,NaN,...,4106.284664,18.461044,-71.538956,44.172180,0.010622,4246.592757,4071.638524,13.395357,-76.604643,NaN


In [16]:
df_test.to_csv('Bird_Test_df.csv')

In [17]:
df_test=df_test[0:0]